# 01 — Dataset (Roboflow) ➜ Filtre *frelon* (COCO 1 classe)

- Télécharge le dataset via **Roboflow** (COCO format).
- Filtre les annotations pour ne garder que la classe **frelon**.
- Sauvegarde des JSON COCO filtrés (train/valid/test).

In [1]:
import os, json
from pathlib import Path
from roboflow import Roboflow


rf = Roboflow(api_key="Z3PKQ49nKlyZJKJems0e")
project = rf.workspace("frelonproject").project("hornet-bees-6bl4k")
version = project.version(2)
dataset = version.download("coco")
download_path = Path(getattr(dataset, "location", str(dataset)))
print(f"[Roboflow] Downloaded to: {download_path}")

DATA_DIR = Path(dataset.location)
TARGET = "frelon"

loading Roboflow workspace...
loading Roboflow project...
[Roboflow] Downloaded to: c:\Users\sBTvR\Documents\Projets techniques\GitHub\Frelon-v0\notebooks\hornet-bees-2


In [2]:
def filter_coco(coco_json_path, keep_class_name, out_json_path):
    with open(coco_json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    name_to_id = {c["name"]: c["id"] for c in coco["categories"]}
    if keep_class_name not in name_to_id:
        raise ValueError(f"Classe '{keep_class_name}' absente de {coco_json_path}.")

    keep_id = name_to_id[keep_class_name]
    anns = [a for a in coco["annotations"] if a["category_id"] == keep_id]
    img_ids = {a["image_id"] for a in anns}
    imgs = [im for im in coco["images"] if im["id"] in img_ids]

    # remap to single-class id=1
    for a in anns: a["category_id"] = 1
    cats = [{"id":1,"name":keep_class_name,"supercategory":keep_class_name}]

    out = {"images": imgs, "annotations": anns, "categories": cats}
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False)
    return len(imgs), len(anns)

In [3]:
for split in ["train","valid","test"]:
    ann = DATA_DIR / split / "_annotations.coco.json"
    out = DATA_DIR / split / f"_annotations_{TARGET}_single.coco.json"
    n_img, n_ann = filter_coco(ann, TARGET, out)
    print(f"{split}: images={n_img}, annots={n_ann}, out={out}")

train: images=922, annots=1017, out=c:\Users\sBTvR\Documents\Projets techniques\GitHub\Frelon-v0\notebooks\hornet-bees-2\train\_annotations_frelon_single.coco.json
valid: images=91, annots=95, out=c:\Users\sBTvR\Documents\Projets techniques\GitHub\Frelon-v0\notebooks\hornet-bees-2\valid\_annotations_frelon_single.coco.json
test: images=45, annots=53, out=c:\Users\sBTvR\Documents\Projets techniques\GitHub\Frelon-v0\notebooks\hornet-bees-2\test\_annotations_frelon_single.coco.json
